In [1]:
from utils import *
from dataset import *
from constants import *
from models.BaseModels import *
from models.DeepModels import *
from models.DeepUtils import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from dataset import Participant
from torch.utils.data import DataLoader

random.seed(RANDOM_STATE)

# Action recognition

In [2]:
part_name = 's6'
use_saved = True

In [3]:
if use_saved and os.path.exists(f'saved/{part_name}.pkl'):
    participant = Participant.load_from_pickle(f'saved/{part_name}.pkl')
else:
    participant = Participant(part_name, data_path=DATA_PATH_NOTEBOOK, alpha=ALPHA)
    saved_dir = os.path.join(os.getcwd(), 'saved')
    if not os.path.exists(saved_dir):
        os.makedirs(saved_dir)
    with open(f'saved/{part_name}.pkl', 'wb') as f:
        pickle.dump(participant, f, pickle.HIGHEST_PROTOCOL)

In [4]:
test_size = 0.3
pca_expl_var = 0.95

<span style='color: red'>WARNING</span>: computing all features takes around 7 minutes (Intel Core i7-7700K)

In [8]:
if use_saved and os.path.exists(f'saved/features_{part_name}_ExObs.h5'):
    features = pd.read_hdf(f'saved/features_{part_name}_ExObs.h5', 'df')
else:
    features = participant.get_features_all_sessions_ExObs()
    features.to_hdf(f'saved/features_{part_name}_ExObs.h5', 'df', mode='w', data_columns=True)

100%|██████████| 256/256 [06:31<00:00,  1.53s/it]
C:\Users\sarah\AppData\Local\Temp\ipykernel_12308\1118112396.py:5: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  features.to_hdf(f'saved/features_{part_name}_ExObs.h5', 'df', mode='w', data_columns=True)


In [9]:
print(f'The dataset contains {features.shape[0]} samples and {features.shape[1]} features.')
print(f'The {len(participant.relevant_channels_ex)} relevant channels are located in the following locations:')
regions = [participant.channels_locations[i] for i in [channel.idx for channel in participant.relevant_channels_ex]]
channels_per_regions = {}
for region in set(regions):
    channels_per_regions[region] = regions.count(region)
print(channels_per_regions)

The dataset contains 256 samples and 2881 features.
The 68 relevant channels are located in the following locations:
{'WM_precentral': 6, 'paracentral': 3, 'postcentral': 16, 'superiorfrontal': 5, 'supramarginal': 3, 'WM_paracentral': 1, 'caudalmiddlefrontal': 4, 'precentral': 26, 'insula': 3, 'WM_insula': 1}


Let's create a baseline by taking the same number of channels, but without checking whether they are responsive:

<span style='color: red'>WARNING</span>: computing all features takes around 7 minutes (Intel Core i7-7700K)

In [10]:
if use_saved and os.path.exists(f'saved/baseline_features_{part_name}_ExObs.h5'):
    baseline_features = pd.read_hdf(f'saved/baseline_features_{part_name}_ExObs.h5', 'df')
else:
    baseline_features = participant.get_features_all_sessions_rnd(len(participant.relevant_channels_ex), movtype='E')
    baseline_features.to_hdf(f'saved/baseline_features_{part_name}_ExObs.h5', 'df', mode='w', data_columns=True)

100%|██████████| 256/256 [05:12<00:00,  1.22s/it]
C:\Users\sarah\AppData\Local\Temp\ipykernel_12308\2194626407.py:5: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  baseline_features.to_hdf(f'saved/baseline_features_{part_name}_ExObs.h5', 'df', mode='w', data_columns=True)


In [11]:
print(f'The baseline dataset contains {baseline_features.shape[0]} samples and {baseline_features.shape[1]} features.')

The baseline dataset contains 128 samples and 4897 features.
